In [1]:
from discovery_child_development import config, PROJECT_DIR, S3_BUCKET
from discovery_child_development.getters import taxonomy
from discovery_child_development.utils import jsonl_utils as jsonl
from discovery_child_development.utils import taxonomy_labelling_utils as tlu
from discovery_child_development.utils.openai_utils import client

from nesta_ds_utils.loading_saving import S3 as nesta_s3

import pandas as pd
import random
import tiktoken

pd.set_option('max_colwidth', 400)

MODEL = "gpt-3.5-turbo-1106" # "gpt-4"

MODEL_INPUT_COST, MODEL_OUTPUT_COST = tlu.get_model_cost(MODEL)
SEED = config["seed"]

random.seed(SEED)

encoding = tiktoken.encoding_for_model(MODEL)

In [2]:
data_to_be_labelled = pd.DataFrame(taxonomy.get_labelling_sample())
data_to_be_labelled.head()

2023-12-20 16:19:05,221 - botocore.credentials - INFO - Found credentials in environment variables.
2023-12-20 16:19:07,168 - root - INFO - File data/labels/taxonomy_classifier/training_validation_data_patents_openalex.jsonl downloaded from discovery-iss to /Users/rosie.oxbury/Documents/git_repos/discovery_child_development/inputs/data/labelling/taxonomy/input/training_validation_data_patents_openalex.jsonl


,id,text,source
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex
3,W1538577929,"Experience-based language acquisition: a computational model of human language acquisition. Almost from the very beginning of the digital age, people have sought better ways to communicate with computers. This research investigates how computers might be enabled to understand natural language in a more humanlike way. Based, in part, on cognitive development in infants, we introduce an open com...",openalex
4,W1556618260,"A new old age?: Exploring the values, attitudes and expectations of baby boomers and their implications for policy and practice in an ageing society. &lt;p&gt;As a large age cohort, baby boomers needs, attitudes and behaviour can have a substantial impact on society. Despite international predictions that older boomers will behave in very different ways to current generations of older people l...",openalex


In [3]:
len(data_to_be_labelled) - len(data_to_be_labelled['id'].unique())

0

In [4]:
test_sample = data_to_be_labelled.head(30)

In [5]:
test_sample

,id,text,source
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex
3,W1538577929,"Experience-based language acquisition: a computational model of human language acquisition. Almost from the very beginning of the digital age, people have sought better ways to communicate with computers. This research investigates how computers might be enabled to understand natural language in a more humanlike way. Based, in part, on cognitive development in infants, we introduce an open com...",openalex
4,W1556618260,"A new old age?: Exploring the values, attitudes and expectations of baby boomers and their implications for policy and practice in an ageing society. &lt;p&gt;As a large age cohort, baby boomers needs, attitudes and behaviour can have a substantial impact on society. Despite international predictions that older boomers will behave in very different ways to current generations of older people l...",openalex
5,W1779055190,"The influence of the workplace environment on breastfeeding practices of working mothers returning to work : a case study of two companies in KwaZulu-Natal. Purpose: Breastfeeding is a key child survival strategy important for the long-term health of both the mother and child. The number of women in paid employment has increased exponentially, yet very few of these women continue breastfeeding...",openalex
6,W2153822755,"Mapping the Early Intervention System in Ontario, Canada. &lt;p&gt;This study documents the wide range of early intervention services across the province of Ontario. The services are mapped across the province showing geographic information as well as the scope of services (clinical, family-based, resource support, etc.), the range of early intervention professionals, sources of funding and th...",openalex
7,W2221947493,Exposure to Electronic Entertainment Media and Student Outcomes in Two Demographically Diverse Elementary Schools. Problem. Standardized test-score results have declined sharply and learning difficulties have risen significantly during the decades that electronic entertainment media (EEM) have become a national phenomenon. Exposure to electronic entertainment media has generated much speculati...,openalex
8,W2276592165,"City Management Theory and Practice: A Foundation for Educating the Next Generation of Local Government Administrators. Due to the retirement of the baby boom generation, many individuals, in a relatively short period of time, will need to be trained and educated to step into local government senior leadership positions. Ideally, the education and training of this new leadership cadre should b...",openalex
9,W2282897564,"Research as a tool. This chapter demystifies research and the research process, drawing on authors' combined knowledge and experience of doing early childhood research, and t

In [6]:
categories_flat = tlu.load_categories()

function = tlu.format_function(categories_flat)

In [7]:
labelled_data = []
    
for index, row in test_sample.iterrows():
    prompt = tlu.build_prompt(row['text'], categories_flat)
    r = client.chat.completions.create(
        model=MODEL,
        temperature=0.0,
        messages=prompt,
        functions=[function],
        function_call={"name": "predict_category"},
        )
    llm_output = tlu.get_labels_from_gpt_response(r)
    output = {"id": row['id'],
            "text": row["text"],
            "source": row["source"],
            "labels": llm_output,
            "tokens_input": r.usage.prompt_tokens,
            "tokens_output": r.usage.completion_tokens,
            "cost": (
                MODEL_INPUT_COST * (r.usage.prompt_tokens / 1000)
            ) + (MODEL_OUTPUT_COST * (r.usage.completion_tokens / 1000))}
    labelled_data.append(output)

2023-12-20 16:19:09,470 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:11,301 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:13,316 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:14,577 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:16,078 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:18,513 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:20,843 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:24,541 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-20 16:19:26,690 - httpx 

In [8]:
labelled_data = pd.DataFrame(labelled_data)

In [9]:
labelled_data['cost'].sum()

0.046706000000000004

In [10]:
labelled_data = labelled_data.explode('labels')

labelled_data.head()

,id,text,source,labels,tokens_input,tokens_output,cost
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex,SEND,1643,5,0.001653
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Cognitive development,1229,19,0.001267
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Communication and language,1229,19,0.001267
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Early childhood development (general),1229,19,0.001267
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Education,1229,19,0.001267


In [11]:
labelled_data[labelled_data['id']=='W1556618260']['labels']

4    Policy
Name: labels, dtype: object

In [12]:
category_names = list(categories_flat.keys())

In [13]:
new_category_dict = tlu.make_keyword_dict(categories_flat)

In [14]:
labelled_data['category'] = labelled_data['labels'].apply(lambda x: tlu.map_keywords_to_categories(x, new_category_dict, category_names))

labelled_data_cleaned = labelled_data[labelled_data['category']!='no label']

In [15]:
labelled_data.head(50)

,id,text,source,labels,tokens_input,tokens_output,cost,category
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex,SEND,1643,5,0.001653,SEND
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Cognitive development,1229,19,0.001267,Cognitive development
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Communication and language,1229,19,0.001267,Communication and language
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Early childhood development (general),1229,19,0.001267,Early childhood development (general)
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Education,1229,19,0.001267,Education
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,Expressive arts and design,1436,24,0.001484,Expressive arts and design
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,Communication and language,1436,24,0.001484,Communication and language
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,Personal social emotional,1436,24,0.001484,Personal social emotional
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how m

In [16]:
len(labelled_data) - len(labelled_data_cleaned)

25

In [17]:
labelled_data[labelled_data['category'].str.lower()!=labelled_data['labels']]

,id,text,source,labels,tokens_input,tokens_output,cost,category
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex,SEND,1643,5,0.001653,SEND
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Cognitive development,1229,19,0.001267,Cognitive development
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Communication and language,1229,19,0.001267,Communication and language
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Early childhood development (general),1229,19,0.001267,Early childhood development (general)
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Education,1229,19,0.001267,Education
...,...,...,...,...,...,...,...,...
27,W2802848482,"Maternal stress during pregnancy and early childhood development. We estimate the impact of prenatal stress on early childhood development outcomes known as ""middle years"" or intermediate outcomes, which has not been studied previously. Using a unique measure of actual maternal stress induced by a large earthquake, we find that relative to children that were not exposed, in utero maternal stre...",openalex,Prenatal,1303,17,0.001337,Prenatal
27,W2802848482,"Maternal stress during pregnancy and early childhood development. We estimate the impact of prenatal stress on early childhood development outcomes known as ""middle years"" or intermediate outcomes, which has not been studied previously. Using a unique measure of actual maternal stress induced by a large earthquake, we find that relative to children that were not exposed, in utero maternal stre...",openalex,Early childhood development (general),1303,17,0.001337,Early childhood development (general)
27,W2802848482,"Maternal stress during pregnancy and early childhood development. We estimate the impact of prenatal stress on early childhood development outcomes known as ""middle years"" or intermediate outcomes, which has not been studied previously. Using a unique measure of actual maternal stress induced by a large earthquake, we find that relative to children that were not exposed, in utero maternal stre...",openalex,Cognitive development,1303,17,0.001337,Cognitive development
28,W2809913025,"Concurrent exposure to heavy metals and cognition in school-age children in Congo-Kinshasa: A complex 

In [18]:
labelled_data[labelled_data['category']=='no label']

,id,text,source,labels,tokens_input,tokens_output,cost,category
9,W2282897564,"Research as a tool. This chapter demystifies research and the research process, drawing on authors' combined knowledge and experience of doing early childhood research, and training research students over many years. It emphasises that research is best conceptualised simply as a tool that helps to answer important questions about early childhood questions that would remain unanswered were it n...",openalex,Research,1353,23,0.001399,no label
9,W2282897564,"Research as a tool. This chapter demystifies research and the research process, drawing on authors' combined knowledge and experience of doing early childhood research, and training research students over many years. It emphasises that research is best conceptualised simply as a tool that helps to answer important questions about early childhood questions that would remain unanswered were it n...",openalex,Child development,1353,23,0.001399,no label
13,W2588700166,Precursors To Engineering Thinking (Pet). Abstract NOTE: The first page of text has been automatically extracted and included below in lieu of an abstract Precursors to Engineering Thinking (PET) Project: Intentional Designs with Experimental Artifacts (IDEA) Introduction Children’s play naturally employs skills of observation and experimentation that lead to the development of intuitive model...,openalex,Cognition,1737,64,0.001865,no label
25,W2794497562,"Elementary Educators' Attitudes about the Utility of Educational Robotics and Their Ability and Intent to Use It with Students. Educational robotics (ER) combines accessible and age-appropriate building materials, programmable interfaces, and computer coding to teach science and mathematics using the engineering design process. ER has been shown to increase K-12 students' understanding of STEM...",openalex,STEM,1818,57,0.001932,no label
25,W2794497562,"Elementary Educators' Attitudes about the Utility of Educational Robotics and Their Ability and Intent to Use It with Students. Educational robotics (ER) combines accessible and age-appropriate building materials, programmable interfaces, and computer coding to teach science and mathematics using the engineering design process. ER has been shown to increase K-12 students' understanding of STEM...",openalex,Science,1818,57,0.001932,no label
25,W2794497562,"Elementary Educators' Attitudes about the Utility of Educational Robotics and Their Ability and Intent to Use It with Students. Educational robotics (ER) combines accessible and age-appropriate building materials, programmable interfaces, and computer coding to teach science and mathematics using the engineering design process. ER has been shown to increase K-12 students' understanding of STEM...",openalex,Professional development,1818,57,0.001932,no label
25,W2794497562,"Elementary Educators' Attitudes about the Utility of Educational Robotics and Their Ability and Intent to Use It with Students. Educational robotics (ER) combines accessible and age-appropriate building materials, programmable interfaces, and computer coding to teach science and mathematics using the engineering design process. ER has been shown to increase K-12 students' understanding of STEM...",openalex,Engineering,1818,57,0.001932,no label
25,W2794497562,"Elementary Educators' Attitudes about the Utility of Educational Robotics and Their Ability and Intent to Use It with Students. Educational robotics (ER) combines accessible and age-appropriate building materials, programmable interfaces, and computer coding to teach science and mathematics using the engineering design process. ER has been shown to increase K-12 students' understanding of STEM...",openalex,Classroom,1818,57,0.001932,no label
25,W2794497562,"Elementary Educators' Attitudes about the Utility of Educational Robotics and Their Ability and Intent to Use It with Students. Educational robotics (ER) combines accessible and age-appropriate building material

In [19]:
labelled_data_cleaned[labelled_data_cleaned['id']=='W1556618260']['category']

4    Policy
Name: category, dtype: object

In [20]:
nesta_s3.upload_obj(labelled_data_cleaned, S3_BUCKET, "data/labels/taxonomy_classifier/labelled_with_gpt/training_validation_data_patents_openalex_GPT_LABELLED.parquet")

In [21]:
test_df = taxonomy.get_gpt_labelled_sample()

In [23]:
test_df.head(30)

,id,text,source,labels,tokens_input,tokens_output,cost,category
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex,SEND,1643,5,0.001653,SEND
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Cognitive development,1229,19,0.001267,Cognitive development
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Communication and language,1229,19,0.001267,Communication and language
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Early childhood development (general),1229,19,0.001267,Early childhood development (general)
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,Education,1229,19,0.001267,Education
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,Expressive arts and design,1436,24,0.001484,Expressive arts and design
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,Communication and language,1436,24,0.001484,Communication and language
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,Personal social emotional,1436,24,0.001484,Personal social emotional
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how m

In [33]:
t = nesta_s3.download_obj(S3_BUCKET, "data/labels/taxonomy_classifier/labelled_with_gpt/training_validation_data_patents_openalex_GPT_LABELLED_test.parquet", download_as="dataframe"
        )

In [34]:
t.head()

,id,text,source,cost,labels_raw,labels
0,W1519846160,"Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edward...",openalex,0.001653,[SEND],[SEND]
1,W1523874198,"Early Bilingual Education. In the coming years, an increasing number of children in urban Chinese areas are taking a second language at an early age. It leads to an argument about whether it is appropriate for children to learn a second language in an early age Some people disagree that idea, because children are not developed enough to receive foreign language learning.",openalex,0.001243,[Cognitive development],[Cognitive development]
2,W1535586210,The Use of Music Therapy Strategies to Enhance the General Milieu in a Rehabilitation Centre. &lt;p&gt;This research focused on how music therapy strategies could be used to improve the milieu in a rehabilitation centre in a New Zealand city. Residents in the rehabilitation centre had difficulty interacting with each other. I planned to use a Community Music Therapy approach to address residen...,openalex,0.001486,"[Health, Communication and language, Early childhood development (general), Expressive arts and design, Personal social emotional]","[Health, Communication and language, Early childhood development (general), Expressive arts and design, Personal social emotional]"
3,W1538577929,"Experience-based language acquisition: a computational model of human language acquisition. Almost from the very beginning of the digital age, people have sought better ways to communicate with computers. This research investigates how computers might be enabled to understand natural language in a more humanlike way. Based, in part, on cognitive development in infants, we introduce an open com...",openalex,0.001655,"[Communication and language, Technology (general), Data Science and AI, Cognitive development]","[Communication and language, Technology (general), Data Science and AI, Cognitive development]"
4,W1556618260,"A new old age?: Exploring the values, attitudes and expectations of baby boomers and their implications for policy and practice in an ageing society. &lt;p&gt;As a large age cohort, baby boomers needs, attitudes and behaviour can have a substantial impact on society. Despite international predictions that older boomers will behave in very different ways to current generations of older people l...",openalex,0.001783,"[Education, Social environment, Culture and communities, Labour market, Policy, Family environment]","[Education, Social environment, Culture and communities, Labour market, Policy, Family environment]"
